# Business Understanding

## なにをするのか

ビジネス上発生している問題を機械学習モデルの予測がどのように解決するのか、関係者へのヒアリングを通じ理解しBusiness goalを設定します。

## なぜやるのか

[機械学習活用プロジェクトの80%は失敗するといわれています](https://research.aimultiple.com/ai-fail/)。最も多い理由の一つとして、ビジネス要件が不明確であったことが挙げられてます。

1. Gartner
   * 2022年までに85%のAI活用プロジェクトが失敗すると予想
2. MIT Sloan Management Review, Boston Consulting Group Artificial Intelligence Global Executive Study
   * 2019年の調査で、65%の企業は投資の効果を実感していない。
3. Transform 2019
   * データサイエンスプロジェクトの87%は失敗する。

機械学習モデルの開発には様々なロールが関わります。関係するロール(ステークホルダー)の間で、機械学習モデルの価値を合意する必要があります。

## どうやるのか

機械学習モデルの開発により解決できる課題のインパクトと、モデルの使用頻度がモデルの価値を決めます。インパクト、使用頻度ともに高く実現性が高い開発が最優先です。モデルの使用頻度が大きいほど、課題解決のインパクトに占める機械学習モデルの貢献が大きくなります。

* 課題解決のインパクト: 問題のあるAs-Isと、To-beとして目指したいギャップが大きいほどインパクトが大きい。
* 機械学習モデルの使用頻度: 意思決定の選択肢が多く、意思決定の機会が多いほど使用頻度が高い。
* 課題解決の実現性: 先行事例や研究が存在する。また、実現するためのリソース(人材、予算、データ)が調達可能であるほど実現性が高い。

今回のハンズオンは次のようなシナリオを想定します。

* 住宅を販売する会社HouseCompanyで機械学習モデルを開発するプロジェクトが立ち上がり、データサイエンティストが採用された。
* HouseCompanyの売上を拡大するには取り扱い地域の拡大による販売戸数の拡大が不可欠である。
* 販売する住宅の情報収集は社内のだれでも行うことができるが、住宅の価格設定は少数の業務有識者の経験が必要である。
* 業務有識者の数がボトルネックとなり、販売戸数の拡大には限界がある。
* 機械学習モデルには、業務有識者の代替か生産性向上が期待されている。

シナリオに沿った分析例は次のようになります。

* 課題解決のインパクト分析
   * As-Is: 住宅の価格設定は少数の業務有識者に依存しており、販売戸数拡大のボトルネックとなっている。
   * To-be: 販売する住宅の情報収集と同じスピードで住宅の価格設定も行える。
   * Impact: 販売戸数をN件/年からM件/年に増やすことができ、売上をX%拡大できる。
* 機械学習モデルの使用頻度分析
   * 選択肢の数: HouseCompanyで取り扱う住宅の価格は主に10万ドルから30万ドルの範囲で、1000ドル刻み程度(200種類程度)。
   * 意思決定の回数: 住宅の値付けは月にN件程度だが、増やしたいとは何年も前から言われている。
   * Impact: 200種類 * N件 * 人件費/件 = X円
* 課題解決の実現性分析
   * 先行事例: あり。
   * 人材: データサイエンティストを採用。
   * 予算: Xドル。隔週で値付け担当者にミーティングへ参加してもらえることを確約済み。
   * データ: データベースにこれまでの住宅情報、価格のデータが格納されている。

機械学習モデルの開発を始める前に、候補となる案件をこの3つの観点で整理すると優先順位がつけやすくなります。
Business Goalは、「課題解決のインパクトの達成」「機械学習モデルの使用頻度の達成」の2段階で設定するとよいでしょう。機械学習モデルがうまく利用されていても何らかの外部要因で課題が解決できないことがあるため、分離して評価すべきです。

Business goal

* Solution Impact: 売上をX%拡大する。
* Model Usage Impact: X円かかっている1件当たりの住宅価格算定業務をY円に削減する。

### プログラミング演習

(Business UnderstandingではTechnical Exerciseはありません)

### コミュニケーション演習

データサイエンティストとして、Business Understandingを行うには誰にどのような質問をすべきでしょうか。

* **課題解決のインパクト**分析
* 機械学習モデルの**使用頻度**分析
* 課題解決の実現性分析


| 英語名             | 日本語名                   | 役割                                                                 |
|--------------------|----------------------------|------------------------------|
| Product Manager    | プロダクトマネージャー     | ユーザー体験を向上させるために実装すべきソフトウェア機能を定義する。 |
| Business Analyst   | ビジネスアナリスト         | 解決すべきビジネス上の問題を定義する                                 |
| Data Analyst       | データアナリスト           | データの可視化と分析で問題を定量的に特定する。                       |
| Architect          | アーキテクト               | ソフトウェアアーキテクチャ全体を設計する。                           |
| DevOps Engineer    | DevOpsエンジニア           | ソフトウェアの開発・運用プロセスを自動化する。                       |
| Software Engineer  | ソフトウェアエンジニア     | ソフトウェアの開発を行う。                                           |
| Operator           | 業務担当者                 | ソフトウェアを利用し業務を行う。                                     |
| System Admin       | システム管理者             | サービスの挙動を監視する。                                           |
| IT Auditor         | IT監督者                   | システムや会社全体の権限管理や監査を行う。                           |
| Data architect     | データアーキテクト         | データを管理する基盤を設計、運用する。                               |
| Domain Expert      | 業務有識者                 | 深い業務知識を持ちデータの意味やあるべき状態について定義する。       |
| Data Scientist     | データサイエンティスト     | 機械学習モデルを開発する。                                           |

#### 質問例

* Product Manager/Business Analyst
  * As-Isの問題
  * To-beの課題。経営計画などの資料があればよいインプットになります。
  * これまで実施された課題解決のための施策
* Domain Expert
  * 選択肢の数
  * 意思決定の回数

## Next Step

[Analyze](02_analyze.ipynb)

## References

1. 木田浩理, 伊藤豪, 高階勇人 and 山田紘史. [データ分析人材になる。 目指すは「ビジネストランスレーター」](https://www.amazon.co.jp/dp/B08L4DQ311/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1). 2020.
2. 有賀康顕, 中山心太 and 西林孝. [仕事ではじめる機械学習 第2版](https://www.amazon.co.jp/%E4%BB%95%E4%BA%8B%E3%81%A7%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92-%E7%AC%AC2%E7%89%88-%E6%9C%89%E8%B3%80-%E5%BA%B7%E9%A1%95/dp/4873119472). 2021.
3. 河本薫. [データ分析・AIを実務に活かす データドリブン思考](https://www.amazon.co.jp/dp/B09LQ8W13Y/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1). 2022.
4. 藤井亮宏. [現場で活用するための機械学習エンジニアリング](http://xn--88jmw4a2gqcujwb5dyd3k9nxde6381meykn68cwlghtoc36a5dfiq2e/). 2021.
